In [ ]:
from RTdata import get_data
from indicators import add_indicators
from Trade import go_long, go_short
import time

In [ ]:
data_df = get_data('crypto_bar')

In [ ]:
data_df.head(2)

In [ ]:
# Set 'time' index
data_index_df = data_df.set_index('time')

#Transfer datatype
data_index_df[['symbol','type']]= data_index_df[['symbol','type']].astype('str')
data_index_df[['close', 'high', 'low', 'open','trade','volume','vwap']] = data_index_df[['close', 'high', 'low', 'open','trade','volume','vwap']].astype('float')
data_index_df.dtypes

In [ ]:
# Add indicators.
data_indicators_df = add_indicators(data_index_df)
data_indicators_df.head(2)

In [ ]:
data_indicators_df.dtypes

In [ ]:
# Create data acquire function:
def clean_data():
    data_df = get_data('crypto_bar')
    # Set 'time' index
    data_index_df = data_df.set_index('time')

    #Transfer datatype
    data_index_df[['symbol','type']]= data_index_df[['symbol','type']].astype('str')
    data_index_df[['close', 'high', 'low', 'open','trade','volume','vwap']] = data_index_df[['close', 'high', 'low', 'open','trade','volume','vwap']].astype('float')
    # Add indicators.
    data_indicators_df = add_indicators(data_index_df)
    return data_indicators_df

In [ ]:
data_indicators_df = clean_data()
data_indicators_df[data_indicators_df['BBL_5_2.0'] >= data_indicators_df['close']]
data_indicators_df.head(25)

In [ ]:
'''
1.D％值在70以上时，市场呈现超买现象。D％值在30以下时市场则呈现超卖现象。 
2.当Ｋ％线发生倾斜度趋于平缓时，是为警告讯号。 
3.当K％值大于D％值，显示目前是向上涨升的趋势，因此在图形上K％向上突破D％线时，即为买进的讯号。 
4.当D％值大于Ｋ％值，显示目前的趋势是向下跌落，因此在图形上K％向下跌破D％线，即为卖出讯号。 
5.K％线与D％线的交叉，须在80以下，20以下讯号才较为正确。K％与D％线在特性上与强弱指标一样，当Ｋ％值与D％值在70以上，已显示超买的现象，30以下出现超卖的现象。唯强弱指标不能明显的显示买卖时机，而KD线则可以达到此目的。 
6.背离讯号产生时，亦为非常正确的买进、卖出时机。 
7.发行量太小，交易量太小的股票不适用KD指标；加权指数以及热门大型股准确性极高。 
8.KD值在50左右交叉且为盘局时，此指标不能当成明显的买卖讯号。
9.J％>100时为超买，J％<10时为超卖，反应较K％及D％为快。KD指标是计算ｎ日内买卖超情况，再加上平均线观念而求出的可设定计算期（即ｎ值），P1、P2、P3值。一般P1，P2都设成3，若P3设成0，表示不画J线。 
'''

In [ ]:
# Set Condition to excute trade.
def trade_condition_check(qty):
    while True:
        # get clean data.
        data_indicators_df = clean_data()
        # Go long condition 1 BOLL: close <= BBL
        if data_indicators_df['close'][0] <= data_indicators_df['BBL_5_2.0'][4]:
            # set current price as low price
            low_price_point = data_indicators_df['low']
            print(f'Low price point: {low_price_point}')
            if data_indicators_df['D_9_3'][15] < 20:
                if data_indicators_df['K_9_3'][13] > data_indicators_df['D_9_3'][15]:
                    if data_indicators_df['J_9_3'][15] < 10: 
                        symbol = data_indicators_df['symbol'][0]
                        limit_price = data_indicators_df['low'][0]
                        take_profit_price = limit_price * 1.01
                        stop_loss_price = limit_price * 0.99
                        go_long(symbol,qty,limit_price,take_profit_price,stop_loss_price)
                        print('Trade placed')
                        break
        # Go short condition 1 BOLL: close >= BBL
        elif data_indicators_df['close'][0] >= data_indicators_df['BBU_5_2.0'][4]:
            high_price_point = data_indicators_df['high']
            print(f'High price point: {high_price_point}')
            # Go Short condition 2 - KDJ 
            if data_indicators_df['D_9_3'][15] > 80:
                if data_indicators_df['K_9_3'][13] < data_indicators_df['D_9_3'][15]:
                    if data_indicators_df['J_9_3'][15] > 100:
                        # prepare trade detail
                        symbol = data_indicators_df['symbol'][0]
                        limit_price = data_indicators_df['high'][0]
                        take_profit_price = limit_price * 0.99
                        stop_loss_price = limit_price * 1.01
                        # submit trade
                        go_short(symbol,qty,limit_price,take_profit_price,stop_loss_price)
                        print('Trade placed')
                        break
        else:
            print(f"***Monitoring the trade condition***\n"
                  f"Ticker: {data_indicators_df['symbol'][0]}\n"
                  f"Close: {data_indicators_df['close'][0]}\n"
                  f"High: {data_indicators_df['high'][0]}\n"
                  f"Low: {data_indicators_df['low'][0]}\n"
                  f"BBL: {data_indicators_df['BBL_5_2.0'][4]}\n"
                  f"BBU: {data_indicators_df['BBU_5_2.0'][4]}\n"
                  f"K: {data_indicators_df['K_9_3'][13]}\n"
                  f"D: {data_indicators_df['D_9_3'][15]}\n"
                  f"J: {data_indicators_df['K_9_3'][13]}"
                 )

            
        time.sleep(60)

In [ ]:
trade_condition_check(qty = 1)